<a href="https://colab.research.google.com/github/Estebanarci2/Marketing-analitica3/blob/main/C)MODELOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install scikit-surprise

In [18]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import sqlite3 as sql
import joblib
from ipywidgets import interact
from mlxtend.preprocessing import TransactionEncoder
from sklearn.preprocessing import MinMaxScaler
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from sklearn import neighbors
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
conn = sql.connect ("/content/drive/MyDrive/analitica3/marketing /db_movies_2")
cur=conn.cursor()

In [21]:
cur.execute("Select name from sqlite_master where type='table'")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('usuarios_fil',),
 ('movies_sel',),
 ('movies_fil',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',),
 ('reco',)]

##**Sistema de recomendación basado en popularidad**

In [22]:
####¿Cuáles son las 10 películas con la calificación promedio más alta?
consulta_sql = """
    SELECT title, AVG(rating) as calificacion
    FROM full_ratings
    GROUP BY title
    ORDER BY calificacion DESC
    LIMIT 10
"""

In [23]:
pd.read_sql(consulta_sql, conn)

,title,calificacion
0,"Streetcar Named Desire, A (1951)",4.615385
1,Harold and Maude (1971),4.531250
2,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.529412
3,"River Runs Through It, A (1992)",4.500000
4,Hoop Dreams (1994),4.458333
5,"Shawshank Redemption, The (1994)",4.439716
6,Raging Bull (1980),4.437500
7,Notorious (1946),4.416667
8,"Manchurian Candidate, The (1962)",4.409091
9,Lawrence of Arabia (1962),4.403226


In [24]:
#### ¿Cuáles son las 10 películas más populares (con más calificaciones) junto con el número total de calificaciones que han recibido?
consulta_sql = """
    SELECT title, COUNT(rating) as total_calificacion
    FROM full_ratings
    GROUP BY title
    ORDER BY total_calificacion DESC
    LIMIT 10
"""

In [25]:
pd.read_sql(consulta_sql, conn)

,title,total_calificacion
0,Forrest Gump (1994),287
1,"Shawshank Redemption, The (1994)",282
2,Pulp Fiction (1994),269
3,"Silence of the Lambs, The (1991)",244
4,"Matrix, The (1999)",236
5,Star Wars: Episode IV - A New Hope (1977),211
6,Braveheart (1995),205
7,Jurassic Park (1993),198
8,Schindler's List (1993),192
9,Terminator 2: Judgment Day (1991),188


In [26]:
####¿Cuáles son los 5 géneros más vistos basado en la cantidad total de películas en ese género?
consulta_sql = """
    SELECT SUBSTR(genres, 1, INSTR(genres, '|') - 1) AS genero, COUNT(genres) AS total_peliculas, AVG(rating) as calificacion
    FROM full_ratings
    GROUP BY genero
    ORDER BY calificacion DESC
    LIMIT 5
"""

In [27]:
pd.read_sql(consulta_sql, conn)

,genero,total_peliculas,calificacion
0,Film-Noir,59,4.262712
1,Musical,47,4.127660
2,Crime,3313,4.029279
3,Mystery,595,3.991597
4,Fantasy,98,3.867347


In [28]:
consulta_sql = """
    SELECT * FROM movies_final
"""

In [29]:
pd.read_sql(consulta_sql, conn)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,5,Father of the Bride Part II (1995),Comedy
4,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
1292,148626,"Big Short, The (2015)",Drama
1293,152081,Zootopia (2016),Action|Adventure|Animation|Children|Comedy
1294,164179,Arrival (2016),Sci-Fi
1295,166528,Rogue One: A Star Wars Story (2016),Action|Adventure|Fantasy|Sci-Fi


##**Sistema de recomendación basado en contenido un solo producto - Manual**

In [30]:
#Se asigna la tabla de movies_final a una variable para realizar los modelos
movies=pd.read_sql('select * from movies_final', conn )
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  1297 non-null   int64 
 1   title    1297 non-null   object
 2   genres   1297 non-null   object
dtypes: int64(1), object(2)
memory usage: 30.5+ KB


In [31]:
##### escalar para que año esté en el mismo rango ###

genres=movies['genres'].str.split('|')
te = TransactionEncoder()
genres = te.fit_transform(genres)
genres = pd.DataFrame(genres, columns = te.columns_)
movies.genres.unique()
movies_dum1 = pd.concat([movies, genres], axis=1)

In [32]:
## eliminar filas que no se van a utilizar ###

movies_dum1['year'] = movies_dum1['title'].str.extract(r'\((\d{4})\)')
movies_dum1.info()
movies_dum1['year'].fillna(0, inplace=True)
movies_dum1['year'] = movies_dum1['year'].astype(int)
sc=MinMaxScaler()
movies_dum1[["year"]]=sc.fit_transform(movies_dum1[['year']])
movies_dum1['year'] = movies_dum1['year'].astype(bool)
movies_dum1=movies_dum1.drop(columns=['genres','title', 'movieId'])
movies_dum1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      1297 non-null   int64 
 1   title        1297 non-null   object
 2   genres       1297 non-null   object
 3   Action       1297 non-null   bool  
 4   Adventure    1297 non-null   bool  
 5   Animation    1297 non-null   bool  
 6   Children     1297 non-null   bool  
 7   Comedy       1297 non-null   bool  
 8   Crime        1297 non-null   bool  
 9   Documentary  1297 non-null   bool  
 10  Drama        1297 non-null   bool  
 11  Fantasy      1297 non-null   bool  
 12  Film-Noir    1297 non-null   bool  
 13  Horror       1297 non-null   bool  
 14  IMAX         1297 non-null   bool  
 15  Musical      1297 non-null   bool  
 16  Mystery      1297 non-null   bool  
 17  Romance      1297 non-null   bool  
 18  Sci-Fi       1297 non-null   bool  
 19  Thriller     1297 non-null 

In [33]:
col_dum=genres.columns
movies_dum2=pd.get_dummies(movies_dum1,columns=col_dum)
movies_dum2.shape

(1297, 39)

In [35]:
#### peliculas recomendados ejemplo para visualización todos las peliculas

def recomendacion(pelicula = list(movies['title'])):

    ind_pelicula=movies[movies['title']==pelicula].index.values.astype(int)[0]
    similar_movies=movies_dum2.corrwith(movies_dum2.iloc[ind_pelicula,:],axis=1)
    similar_movies=similar_movies.sort_values(ascending=False)
    top_similar_movies=similar_movies.to_frame(name="correlación").iloc[0:11,]### el 11 es número de movies recomendados
    top_similar_movies['title']=movies["title"] ### agregaro los nombres (como tiene mismo indice no se debe cruzar)

    return top_similar_movies

In [36]:
print(interact(recomendacion))

interactive(children=(Dropdown(description='pelicula', options=('Toy Story (1995)', 'Jumanji (1995)', 'Grumpie…

<function recomendacion at 0x7fcebbf92710>


##**Sistema de recomendación basado en contenido KNN un solo producto visto**

In [37]:
## el coseno de un angulo entre dos vectores es 1 cuando son perpendiculares y 0 cuando son paralelos(indicando que son muy similar324e-06	3.336112e-01	3.336665e-01	3.336665e-es)
model = neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
model.fit(movies_dum2)
dist, idlist = model.kneighbors(movies_dum2)

distancias=pd.DataFrame(dist) ## devuelve un ranking de la distancias más cercanas para cada fila(movie)
id_list=pd.DataFrame(idlist) ## para saber esas distancias a que item corresponde

In [38]:
def MovieRecommender(movie_name = list(movies['title'].value_counts().index)):
    movie_list_name = []
    movie_id = movies[movies['title'] == movie_name].index
    movie_id = movie_id[0]
    for newid in idlist[movie_id]:
        movie_list_name.append(movies.loc[newid].title)
    return movie_list_name


print(interact(MovieRecommender))


interactive(children=(Dropdown(description='movie_name', options=('Toy Story (1995)', 'Short Circuit (1986)', …

<function MovieRecommender at 0x7fcf00798160>


##**Sistema de recomendación basado en contenido KNN Con base en todo lo visto por el usuario**

In [39]:
#Se asigna la consulta de los ususarios a una variale para realizar el modelo
usuarios=pd.read_sql('select distinct (userId) as user_id from ratings_final',conn)

In [41]:
def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):

    ###seleccionar solo los ratings del usuario seleccionado
    ratings=pd.read_sql('select * from ratings_final where userId=:user',conn, params={'user':user_id})

    ###convertir ratings del usuario a array
    l_movie_r=ratings['movieId'].to_numpy()

    ###agregar la columna de movieId y titulo de la película a dummie para filtrar y mostrar nombre
    movies_dum2[['movieId','title']]=movies[['movieId','title']]

    ### filtrar movies calificadas por el usuario
    movies_r=movies_dum2[movies_dum2['movieId'].isin(l_movie_r)]

    ## eliminar columna nombre e ID
    movies_r=movies_r.drop(columns=['movieId','title'])
    movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    ##centroide o perfil del usuario
    centroide=movies_r.groupby("indice").mean()


    ### filtrar movies no vistas
    movie_nr=movies_dum2[~movies_dum2['movieId'].isin(l_movie_r)]
    ## eliminbar nombre e isbn
    movie_nr=movie_nr.drop(columns=['movieId','title'])

    ### entrenar modelo
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(movie_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0] ### queda en un array anidado, para sacarlo
    recomend_b=movies.loc[ids][['title','movieId']]
#    vistos=movies[movies['movieId'].isin(l_movie_r)][['title','movieId']]

    return recomend_b

print(interact(recomendar))

interactive(children=(Dropdown(description='user_id', options=(1, 407, 401, 402, 403, 404, 405, 406, 408, 399,…

<function recomendar at 0x7fcebbf928c0>
